# Introduction
This notebook experiments with the different techniques to segment an image. Since the images that I will be using will mostly have a uniform background, I am trying to find the best technique that gives the best results with the best performance

In [1]:
import cv2
import numpy as np

In [2]:
inputImage = './images/image.png'

# Method 1: Otsu's Method (global thresholding)

In [3]:
def remove_background_1(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use Otsu's thresholding to create a mask
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Refine the mask using morphology
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # Create a 3-channel mask
    mask_inv = cv2.bitwise_not(mask)
    
    # Extract the foreground object
    foreground = cv2.bitwise_and(image, image, mask=mask)
    
    # Convert to transparent background (RGBA)
    b, g, r = cv2.split(foreground)
    alpha = mask  # Use the mask as the alpha channel
    result = cv2.merge([b, g, r, alpha])

    # Save the result
    cv2.imwrite('./images/output1_otsu.png', result)

In [4]:
remove_background_1(inputImage)

# Method 2: Pre-trained background removal using rembg

In [5]:
from rembg import remove
from PIL import Image

def remove_background_2(image_path):
    input_image = Image.open(image_path)
    output_image = remove(input_image)
    output_image.save("./images/output2_rembg.png")


In [6]:
remove_background_2(inputImage)

# Method 3: Canny edge detection

In [7]:
def remove_background_3(image_path):
    edges = cv2.Canny(cv2.imread(image_path), 100, 200)
    cv2.imwrite('./images/output3_cannyedge.png', edges)

In [8]:
remove_background_3(inputImage)

# Method 4: K-means clustering

In [9]:
from sklearn.cluster import KMeans

def remove_background_4(image_path, n_clusters=2):
    image = cv2.imread(image_path)
    pixels = image.reshape((-1, 3))

    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(pixels)
    segmented = kmeans.cluster_centers_[kmeans.labels_].reshape(image.shape).astype(np.uint8)
    cv2.imwrite('./images/output4_kmeans.png', segmented)


In [10]:
remove_background_4(inputImage)